<a href="https://colab.research.google.com/github/MIKRAMS1984/DROO/blob/master/Replicating_DROO__Creating_DNN_for_task_offloading_decision_and_resource_allocation_using_Bisection_optimization_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Research Gaps:


1.   Computational complexity increases due to K quantizaion levels for relaxed binary offloading decision from DNN, although it is Adaptvie but starts from K = N, for each task, where N is the number of wireless devices, max(N = 30) and the number of input_channel_gain samples are 30K.
2.   Acuuracy and Loss(Training Loss and Testing Loss) of DNN is not discussed in the simulation results, nor compared to a benchmark. Only a single graph is depicted, showing the exponential decay of Training Loss w.r.t. time.
3. The convergance rate of Bisection method, used for optimal resource allocations, is slow compared to other optimization methods, such as Newton Raphson. Hence, it increases time complexity.




# Minimum and Maximum values of the dataset_30.
1.   input_h: Min = 7.271724024595681e-05,   Max = 2.977399447439086e-11. ---> Wireless channel gain
2.   output_mode: Min = 0, Max = 1. ---> Computing mode selection
3.   output_a: Min = 0.2924001509337536,   Max=  0.6702989331341545. ---> Time for energy harvesting

4.   output_tau: Min = 0,  Max = 0.7075998490662463 ---> Time for data offloading

5.   output_obj: Min = 3066213.6654264256, Max = 12440743.093937602
 ---> Weighted sum Computaoin Rate.







# 0.Mounting the google drive to display file(data_30.mat) in colab.

In [ ]:
import scipy.io as sio
from google.colab import drive
drive.mount('/content/drive')
S=sio.loadmat('/data_30.mat') #/content/data_30.mat
S

MessageError: ignored

# Getting the minimum and maximum vaules from the dataset_30(Dictionary).

In [ ]:
import numpy as np
o = sio.loadmat('/data_30.mat')['output_a']
m = np.max(o)
mini = np.min(o)
m, mini, 
#/data_30.mat

(0.6702989331341545, 0.2924001509337536)

# 3. Optimization.py

*   Bisection method is used for **optimal** task offloading decision and **resource allocations** (time alloted for energy harvesting and time alloted for task offloading).
*   CD Method is used as a benchmark for comparison of results.


*   def plot_gain(): Plots the computation rate vs time frames.
*   def bisection(): takes input channel gain, computing modes and weights of the wireless devices **as input arguments** and **returns** the maximum computation rate, optimal time for task offloading and energy harvesting.
*   def cd_method(): used to calculate the CPU execution latency for the same dataset for comparison to the bisection method, for 10, 20, and 30 wireless devices.






In [ ]:
import numpy as np
from scipy import optimize
from scipy.special import lambertw
import scipy.io as sio
import time

# Function 'gain_his' to plot Gain ratio (Computation rate) vs Learning steps.

def plot_gian(gain_his):
    import matplotlib.pyplot as plt
    import pandas as pd
    import matplotlib as mpl

    gain_array = np.asarray(gain_his)
    df = pd.DataFrame(gain_his)

    mpl.style.use('seaborn')
    fig, ax = plt.subplots(figsize = (15,8))
    rolling_intv = 20

    plt.plot(np.arange(len(gain_array))+1, df.rolling(rolling_intv, min_periods=1).mean(),'b')
    plt.fill_between(np.arange(len(gain_array)+1,df.rolling(rolling_intv, min_periods=1).min()[0], df.rolling(rolling_intv, min_periods=1).max()[0], color = 'b',alpha = 0.2))
    plt.ylabel('Gain ratio')
    plt.xlabel('learning steps')
    plt.show()

    # Parameters and Equations for Bisectin Method.
    # h = gain
    # M = m from m_list. m_list contains the binary offloading decisions solved by eq.8 and eq.9 in 1.mainPyTorch.
    # weights is empty list.

def bisection(h,M,weights=[]): #used to find avg. time for optimal solution:0.0125s
        o=100                
        p=3
        u=0.7
        eta1=((u*p)**(1.0/3))/o
        ki=10**-26
        eta2=u*p/10**-10
        B=2*10**6
        Vu=1.1
        epsilon=B/(Vu*np.log(2))
        x=[]
        tau_j=a[1:] # a is energy broadcasting time for WDs
                    # tau_j is transmit time for WDs
        # creating M0 and M1 from M.
        M0=np.where(M==0)[0] # Segregating the 0 and 1 binary offloading decisions.
        M1=np.where(M==1)[0] # returns indices.

        hi=np.array([h[i] for i in M0])
        hi=np.array([h[i] for i in M1])

        # if Remainder is 1 weight will be 1.5 else weight will be 1.

        weights = [1.5 if i%2==1 else 1 for i in range(len(M))]

        # Weights for ith and jth Wireless devices.

        wi=np.array([weights[M0[i]]for i in range(len(M0))])
        wj=np.array([weights[M1[i]]for i in range(len(M1))])

        def sum_rate(x):
            sum1=sum(wi*eta1*(hi/ki)**(1.0/3)*x[0]**(1.0/3))
            sum2=0
            for i in range(len(M1)):
                sum2+=wj[i]*epsilon*x[i+1]*np.log(1+eta2*hj[i]**2*x[0]/x[i+1])
            return sum1+sum2

        def phi(v,j):
            return 1/(-1-1/(lambertw(-1/(np.exp(1+v/wj[j]/epsilon))).real))

        def p1(v):
            p1=0
            for j in range(len(M1)):
                p1 +=hj[j]**2 * phi(v,j)
            return 1/(1+p1*eta2)

        def Q(v):
            sum1 = sum(wi*eta1*(hi/ki)**(1.0/3))*p1(v)**(-2/3)/3
            sum2 = 0
            for j in range(len(M1)):
                sum2 += wj[j]*hj[j]**2/(1+1/phi(v,j))
            return sum1+sum2*epsilon*eta2-v

        def tau(v,j):
            return eta2*hj[j]**2*p1(v)*phi(v,j)

        #Bisection Algorithm starts here

        delta=0.005
        UB = 99999999
        LB = 0
        while UB -LB > delta:
            v = (float(UB)+LB)/2
            if Q(v) > 0:
                LB = v
            else:
                UB = v

        x.append(p1(v))
        for j in range(len(M1)):
            x.appned(tau(v,j))

        return sum_rate(x), x[0], x[1:] # Bisection returning computation rate, reward (sum_rate(x)) for r_list.

        # CD Method for Comparison.
        # CD Method Defined.

        def cd_method(h):
            N = len(h) # N = 30
            # M0 is decision mode having 30 saclar values.(0/1)
            M0 = np.random.randint(2,size = N) # 2 gives only binary values in the array. 
            # Input arguments: input channel gain h and offloading decision M0. 
            # bisection returns computation rate 'gain0', time for enery broadcasting 'a' and data offloading time'Tj'.
            gain0,a,Tj = bisection(h,M0) # Calling bisecion fun. 
            g_list = [] # computaion rate for M0 (local computing)
            M_list = [] # offloading_decisions list
            while True:
                for j in range(0,N): # j from 0 to 29
                    M = np.copy(M0)
                    M[j] = (M[j]+1)%2 # M[j] stores values 0/1 for offloading decision.
                    gain,a,Tj = bisection(h,M) # Again calling bisection
                    g_list.append(gain)
                    M_list.append(M)
                g_max = max(g_list) # Maximum reward
                if g_max > gain0: # Comparing the rewards(compuation rate) for M and M0
                    gain0 = g_max
                    M0 = M_list[g_list.index(g_max)]
                else:
                    break
            return gain0,M0 # Returning weighted sum computation rate for task performed locally.

      #  if__name__=="__main__":
            # input channel gain(h) has 10 values.
            h=np.array([6.06020304235508*10**-6,1.10331933767028*10**-5,1.00213540309998*10**-7,1.21610610942759*10**-6,1.96138838395145*10**-6,1.71456339592966*10**-6,5.24563569673585*10**-6,5.89530717142197*10**-7,4.07769429231962*10**-6,2.88333185798682*10**-6])
            M=np.array([1,0,0,0,1,0,0,0,0,0]) # 10 binary offloading decisions.
#    h=np.array([1.00213540309998*10**-7,1.10331933767028*10**-5,6.06020304235508*10**-6,1.21610610942759*10**-6,1.96138838395145*10**-6,1.71456339592966*10**-6,5.24563569673585*10**-6,5.89530717142197*10**-7,4.07769429231962*10**-6,2.88333185798682*10**-6])
#    M=np.array([0,0,1,0,1,0,0,0,0,0])

    
#    h = np.array([4.6368924987170947*10**-7,	1.3479411763648968*10**-7,	7.174945246007612*10**-6,	2.5590719803595445*10**-7,	3.3189928740379023*10**-6,	1.2109071327755575*10**-5,	2.394278475886022*10**-6,	2.179121774067472*10**-6,	5.5213902658478367*10**-8,	2.168778154948169*10**-7,	2.053227965874453*10**-6,	7.002952297466865*10**-8,	7.594077851181444*10**-8,	7.904048961975136*10**-7,	8.867218892023474*10**-7,	5.886007653360979*10**-6,	2.3470565740563855*10**-6,	1.387049627074303*10**-7,	3.359475870531776*10**-7,	2.633733784949562*10**-7,	2.189895264149453*10**-6,	1.129177795302099*10**-5,	1.1760290137191366*10**-6,	1.6588656719735275*10**-7,	1.383637788476638*10**-6,	1.4485928387351664*10**-6,	1.4262265958416598*10**-6, 1.1779725004265418*10**-6, 7.738218993031842*10**-7,	4.763534225174186*10**-6])
#    M =np.array( [0,	0,	1,	0, 0,	1,	0,	0,	0,	0,	0,	0,	0,	0,	0,	1,	0,	0,	0,	0,	0,	1,	0,	0,	0,	0,	0,	0,	0,	1,])
    
#    time the average speed of bisection algorithm
#    repeat = 1
#    M =np.random.randint(2, size=(repeat,len(h)))
#    start_time=time.time()
#    for i in range(repeat):
#        gain,a,Tj= bisection(h,M[i,:])
#    total_time=time.time()-start_time
#    print('time_cost:%s'%(total_time/repeat))

            # input argurments: input channel gain h and offloading decision M0. 
            # bisection returns computation rate 'gain', time for enery broadcasting 'a' and data offloading time'Tj'.
            gain,a,Tj = bisection(h,M) # Calling bisection fun, returning resources allocation(gain, energy broadcasting time and data transmit time).
            print('y:%s'%gain) # Gain(Comutation rate) print
            print('a:%s'%a)    # Energy broadcasting parameter (time alloted for energy harvesting)
            print('Tj:%s'%Tj)  # Offloading time alloted.

            # Calling CD fun.
            # input argurments: input channel gain h. 
            # returning gain0 and computing mode selection M0 (local compute or offload)
            gain0,M0 = cd_method(h) 
            print('max y:%s'%gain0)
            print(M0)

            K=[10, 20, 30] # Number of wireless devices.
            N = 1000 # Iteration numbers to pick the input channel gain value from input_h.

        for k in K:
            channel = sio.loadmat('./data_%d' %int(k))['input_h'] # Input channel gains
            gain = sio.loadmat('./data_%d' %int(k))['output_obj'] # Weighted sum computaion rate.

            start_time = time.time()
            gain_his = [] # To store computation rate
            gain_his_ratio = [] # To get Normalized computation rate= optimal computation rate / max(optimal computation rate) for all 2^N.
            mode_his = []
            for i in range(N): # i from 0 to 999.
                print("%0.if"%(i/N)) # Getting integer values from floating numbers.

                i_idx = i # Saving index values from 0 to 999 for N = 1000.

                h = channel[i_idx,:] # h = from channel[0, :] to channel[999, :]

                # Calling CD fun.
                # input arguments: input channel gain h. 
                # returning gain0 and computing mode selection M0 (local compute or offload)
                gain0, M0 = cd_method(h) 

                gain_his.append(gain0) # appending the outcome in the computation_rate list.
                gain_his_ratio.append(gain_his[-1] / gain[i_idx][0]) # computation rate ratio.
                mode_his.append(M0) # Appending computing mode decision in mode_his list.

                total_time = time.time()-start_time # Caluclating the time for up to 30 Wireless devices.
                print('time_cost:%s' %total_time)
                print('average time per channel:%s'%(total_time/N))

                plot_gain(gain_his_ratio) # Plotting computation rate ratio

                print("gain/max ratio:", sum(gain_his_ratio)/N)   

# 2.DNN Parameters and Structure. 
# memoryPyTorch.py


1.   Creates a class named MemoryDNN to intialize the hyper-parameters of DNN, define the DNN model and activation functions.
2.   def remember(): to replace the oldest memory with the newest.
3. def encode(): to encode the entery, input arguments are channel gain and memory size.
4. def learn(): collecting a batch of random samples from the memory, declaring tensor size to train the model.
5. Model training: apply optimizer, apply loss fun, clear the gradients, perform predictions, check loss, apply backward propagation to minimize the loss by updating weights and biases of DNN, again apply optimizer, calculate cost of loss function and append it to the list named 'cost_his'.
6. def decode(): to have batch dimensions when feeding into Tensor.
7. Perform DNN testing, while checking the quanitzation mode is order preserving 'OP'or 'KNN'.
8. Solve eq.8 and eq.9 using def knm(): to get binary offloading decisions for k = 1 and k > 1 for 'OP' quantization method. These decisions are appended in m_list. Similarly, def knn(): to get binary offloading decisions for 'KNN' quantization method. 
9. def plot_cost(): Plots training loss vs time.



In [ ]:
# 2.a.
import torch.nn as nn

class MemoryDNN:
  def __init__(
      self,
      net,
      learning_rate = 0.01,
      training_interval = 10,
      batch_size = 128,
      memory_size = Memory,
      output_graph = False,
  ):
    self.net = net
    self.training_interval = training_interval
    self.lr = learning_rate
    self.batch_size = batch_size
    self.memory_size = memory_size # 1000

    # Stor all binary actions.
    self.enumerate_actions = []

    self.memory_counter = 1

    self.cost_his = [] # Store training cost

    self.memory = np.zeros((self.memory_size, self.net[0]+self.net[-1])) # 1000, 10+10

    self._build_net() # Construc memory network

  def _build_net(self):
    self.model = nn.Sequential(
      nn.Linear(self.net[0], self.net[1]),
      nn.ReLU(),
      nn.Linear(self.net[1], self.net[2]),
      nn.ReLU(),
      nn.Linear(self.net[2], self.net[3]),
      nn.Sigmoid(),
  )  
    
  def remember(self, h, m):
    idx = self.memory_counter % self.memory_size
    self.memory[idx, :] = np.hstack((h,m))

    self.memory_counter += 1

  def encode(self, h, m):
    self.remember(h,m)
    if self.memory_counter % self.training_interval == 0:
      self.learn()

  def learn(self):
    if self.memory_counter > self.memory_size:
      sample_index = np.random.choice(self.memory_size, size = self.batch_size)
    else:
      sample_index = np.random.choice(self.memory_counter, size = self.batch_size)
    batch_memory = self.memory[sample_index, :]

    h_train = torch.Tensor(batch_memory[:, 0:self.net[0]])
    m_train = torch.Tensor(batch_memory[:, 0:self.net[0]:])

    optimizer = optim.Adam(self.model.parameters(), lr=self.lr, betas = (0.09, 0.999), weight_decay = 0.0001)

    criterion = nn.BCELoss()

    self.model.train()

    optimizer.zero_grad()

    prdict = self.model(h_train)

    loss = criterion(predict, m_train)

    loss.backward()

    optimizer.step()

    self.cost = loss.item()

    assert(self.cost > 0)

    self.cost_his.append(self.cost)

  def decode(self, h, k=1, mode='OP'):
    h=torch.Tensor(h[np.newaxis,:])
    self.model.eval()
    m_pred=self.model(h)
    m_pred = m_pred.dtach().numpy()

    if mode == 'OP':
      return self.knm(m_pred[0],k)
    elif mode == 'KNN':
      return self.knn(m_pred[0],k)
    else:
      print("The action selection must be 'OP' or 'KNN'.")

  def knm(self, m, k=1):
    m_list = []
    m_list.append(1*(m>0.5))

    if k>1:
      m_abs = abs(m-0.5)
    idx_list = np.argsort(m_abs)[:k-1]
    for i in range(k-1):
      if m[idx_list[i]] > 0.5:
        m_list.append(1*(m-m[idx_list[i]]>0))
      else:
        m_list.append(1*(m-m[idx_list[i]] >= 0))
    return m_list

  def knn(self, m, k=1):
    if len(self.enumerate_actions) == 0:
      import itertools
      self.enumerate_actions=np.array(list(map(list, itertools.product([0,1], repeat=self.net[0]))))
      sqd = ((self.enumeate_actions-m)**2).sum(1)
      idx = np.argsort(sqd) # performs indices ascending sorting
      return self.enumeate_actions[idx[:k]]

  def plot_cost(self):
    import matplotlib.pyplot as plt
    plt.plot(np.arange(len(self.cost_his))*self.training_interval, self.cost_his)
    plt.ylabel('Training Loss')
    plt.xlabel('Time Frames')
    plt.show()

# 1.Main DROO algo.
1.   Plots the computation rate vs time frames.
2.   Sets the hyper-parameters for DNN and loads data samples from the datasets.
3. Splits data set as traninig dataset(80%) and testing dataset(20%).
4. Creates an object named 'mem' for MemoryDNN, DNN consists of one input layer having 'N' number of neurons, where N is number of wireless devices. Two hidden layers having 120 and 80 neurons repectively. Finally, output layer consists of N number of neurons.
5. ReLU activation function is used for input and hidden layers, whereas sigmoid activation function is used at output layer, to get relaxed binary offloading decision.
6. An algorithm is written for adaptive K settings as given in the research article.
7. mem.decode function is called to get the binary offloading decisions(m_list) by solving eq.8 and eq.9 of the research article.
8. Bisection function is called to get the reward list(r_list) for all possible computing modes.
9. Then encode function is called to encode the mode with largest reward(highest computation rate).
10. The DROO training ends here. Maximum computation rate, rate ratio, adaptive k value and time are memorized to plot the graphs.




In [ ]:
import scipy.io as sio
import numpy as np
#from memoryPyTorch import MemoryDNN
#from optimization import bisection
import time

In [ ]:
def plot_rate(rate_his, rolling_intv=50):
  import matplotlib.pyplot as plt
  import pandas as pd
  import matplotlib as mpl
  rate_array = np.asarray(rate_his)
  

In [ ]:
rate_array

In [ ]:
import pandas as pd
df = pd.DataFrame(rate_his)

# Visulaize: Normalized Computation rate vs Time Frames.

In [ ]:
mpl.style.use('seaborn')
fig, ax = plt.subplots(figsize=(15, 8))
plt.plot(np.arange(len(rate_array))+1, np.hstack(df.rolling(rolling_intv, min_periods=1).mean().values), 'b')
plt.fill_between(len(rate_array))+1, np.hstack(df.rolling(rolling_intv, min_periods=1).min()[0].values), 
np.hstack(df.rolling(rolling_intv, min_periods=1).max()[0].values), color =  'b', alpha= 0.2)
plt.ylabel('Normalized Computation Rate')
plt.xlabel("Time Frames")
plt.show()

# Save rate_his(Computation rate) as a file.

In [ ]:
def save_to_text(rate_his, file_path):
  with open(file_path, 'w') as f:
    for rate in rate_his:
      f.write("%s \n" %rate)


# 1.Intializing the parameters fo DNN.

In [ ]:
#if__name__=="__main__":
#K = max(K, K_his[-memory_size]) # memory_size = 1024
N =10
n = 3000
K = N
decoder_mode = 'OP'
Memory = 1024
Delta = 32
#print("#user = %d, #channel = %d, K = %d, decoder = %d, Memory = %d, Delta = %d"%(N, n, K, decoder_mode, Memory, Delta))

# 1.Load data using sio.loadmat()

In [ ]:
import scipy.io as sio
N = 5
channel = sio.loadmat("data_%d"%N)["input_h"]
channel, channel.shape, channel.dtype, channel.size

#1. Rate to plot figures only, not for training DROO

In [ ]:
import scipy.io as sio
N = 5
rate = sio.loadmat("data_%d"%N)["output_obj"]
rate, rate.shape, rate.dtype, rate.size

# 1.Changing floating values between 0 and 1 into values greater than 1, for training DROO.

In [ ]:
channel = channel * 1000000
#channel,
channel.shape, channel.dtype, channel.size

# 1. Generating the train and test data from loaded file data_30.mat

In [ ]:
len(channel)

In [ ]:
split_idx = int(0.8*len(channel))
split_idx, 
#split_idx.shape, 
#split_idx.dtype, 
#split_idx.size

In [ ]:
n = 30000
num_test = min(len(channel)-split_idx, n-int(0.8*n)) #(2000, 6000)
num_test,
#num_test.shape

# Visualizing the Network Parameters.
#To check the parameters, shape and size of a class.
#object_name.attribute_name
#object_name.attribute_name.shape

In [ ]:
mem.batch_size

In [ ]:
mem.memory, mem.memory.shape, 

In [ ]:
mem.net

In [ ]:
mem.training_interval

# 1.Creating 'mem' object using class named MemoryDNN using 2.a.

In [ ]:
import numpy as np
import time
mem = MemoryDNN(net=[N, 120, 80, N],
                learning_rate = 0.01,
                training_interval = 10,
                batch_size = 128,
                memory_size = Memory # 1024
                )
start_time = time.time()



# 1.rate_his used to plot the Normalized computation rate vs Time Frames.

In [ ]:
rate_his = []
rate_his_ratio = []
mode_his = []
k_idx_his = []
K_his = []

# 1.Implementing Adaptive K parameters.

In [ ]:
Delta = 32
for i in range(n):
  if i % (n//10) == 0:
    print(" % 0.1f "% (i/n))
    if i > 0 and i % Delta == 0:
      if Delta > 1: 
        max_k = max(k_idx_his[-Delta:-1]) + 1
      else:
        max_k = k_idx_his[-1]+1
      K = min(max_k+1, N)

In [ ]:
K

# 1.Training and Testing

In [ ]:
if i < n-num_test:
  i_idx = i % split_idx # training
else:
  i_idx = i - n + num_test + split_idx # testing



In [ ]:
h = channel[i_idx, :]
h, h.shape, h.size

# 2. Decoding Fun

In [ ]:
def decode(self, h, k=1, mode='OP'):
  h=torch.Tensor(h[np.newaxis,:])
  self.model.eval()
  m_pred=self.model(h)
  m_pred = m_pred.dtach().numpy()

  if mode == 'OP':
    return self.knm(m_pred[0],k)
  elif mode == 'KNN':
    return self.knn(m_pred[0],k)
  else:
    print("The action selection must be 'OP' or 'KNN'.")

#1. Decoding using "OP (Order-Preserving Quantization)" or "KNN( Kth Nearest Neighbour Quantization Method.)"

In [ ]:
import torch
m_list = mem.decode(h, K, decoder_mode)
r_list = [] # Reward List
for m in m_list:
  r_list.append(bisection(h/1000000, m)[0])




# Encode the mode with largest reward

In [ ]:
mem.encode(h, m_list[np.argmax(r_list)])


# The main code for DROO training ends here

# Metrics for Visulaization, Memorizing the largest reward

In [ ]:
rate_his.append(np.max(r_list))
rate_his_ratio.append(rate_his[-1] / rate[i_idx[0]])


#  Recording the index of largest reward

In [ ]:
k_idx_his.append(np.argmax(r_list))

# Record K in case of adaptive K

In [ ]:
K_his.append(K)
mode_his.append(m_list[np.argmax[r_list]])


# Calculating Simulation time

In [ ]:
total_time = time.time()-start_time
mem.plot_cost() # Fun. call to plot cost
plot_rate(rate_his_ratio)

#Visualize: Average Normalized Compuatation Rate vs Time Consumed 

In [ ]:
print("Averaged normalized computation rate:", sum(rate_his_ratio[-num_test:-1])/num_test)
print("Total time consumed: %s" %total_time)
print("Average time per channel: %s" %(total_time/n))

# Saving data into text files

In [ ]:
save_to_text(k_idx_his, "k_idx_his.txt")
save_to_text(K_his, "K_his.txt")
save_to_text(mem.cost_his, "mem.cost_his.txt")
save_to_text(rate_his_ratio, "rate_his_ratio.txt")
save_to_text(mode_his, "mode_his.txt")

# 2. MemoryPyTorch.py to create DNN.
1. to get rate_his

In [ ]:
#from__future__import print_function
import torch 
import torch.optim as optim
import torch.nn as nn
import numpy as np

print(torch.__version__)

In [ ]:
self.memory

In [ ]:
# Stor all binary actions.
self.enumerate_actions = []

In [ ]:
self.memory_counter = []

In [ ]:
self.cost_his = []

In [ ]:

# Intialize zero memory[h,m]
self.memory = np.zeros((self.memory_size, self.net[0]+self.net[-1]))

In [ ]:
self._build_net()


In [ ]:
def _build_net(self):
  self.model = nn.Sequential(
      nn.Linear(self.net[0], self.net[1]),
      nn.ReLU(),
      nn.Linear(self.net[1], self.net[2]),
      nn.ReLU(),
      nn.Linear(self.net[2], self.net[3]),
      nn.Sigmoid(),
  )  


In [ ]:
def remember(self, h, m):
  idx = self.memory_counter % self.memory_size
  self.memory[idx, :] = np.hstack((h,m))

  self.memory_counter += 1
 

In [ ]:
def encode(self, h, m):
  self.remember(h,m)
  if self.memory_counter % self.training_interval == 0:
    self.learn()



In [ ]:
def learn(self):
  if self.memory_counter > self.memory_size:
    sample_index = np.random.choice(self.memory_size, size = self.batch_size)
  else:
    sample_index = np.random.choice(self.memory_counter, size = self.batch_size)
  batch_memory = self.memory[sample_index, :]
  




In [ ]:
h_train = torch.Tensor(batch_memory[:, 0:self.net[0]])
m_train = torch.Tensor(batch_memory[:, 0:self.net[0]:])



# Train the DNN

In [ ]:
import torch
from torch import optim
optimizer = optim.Adam(self.model.parameters(), lr=self.lr, betas = (0.09, 0.999), weight_decay = 0.0001)

In [ ]:
criterion = nn.BCELoss()

In [ ]:
self.model.train()

In [ ]:
optimizer.zero_grad()

In [ ]:
prdict = self.model(h_train)

In [ ]:
loss = criterion(predict, m_train)

In [ ]:
loss.backward()

In [ ]:
optimizer.step()

In [ ]:
sel.cost = loss.item()

In [ ]:
assert(self.cost>0)

In [ ]:
self.cost_his.append(self.cost)

#Implementing equation 8 and 9, to get k binary offloading decisions.

In [ ]:
def knm(self, m, k=1):
  m_list = []
  m_list.append(1*(m>0.5))

  if k>1:
    m_abs = abs(m-0.5)
    idx_list = np.argsort(m_abs)[:k-1]
    for i in range(k-1):
      if m[idx_list[i]] > 0.5:
        m_list.append(1*(m-m[idx_list[i]]>0))
      else:
        m_list.append(1*(m-m[idx_list[i]] >= 0))

  return m_list


# Calculating L2 Norm using Itertools

In [ ]:
def knn(self, m, k=1):
  if len(self.enumerate_actions) == 0:
    import itertools
    self.enumerate_actions=np.array(list(map(list, itertools.product([0,1], repeat=self.net[0]))))
    sqd = ((self.enumeate_actions-m)**2).sum(1)
    idx = np.argsort(sqd) # performs indices ascending sorting
    return self.enumeate_actions[idx[:k]]

# Visualize: Plotting Training Loss vs Time Frames.

In [ ]:
def plot_cost(self):
  import matplotlib.pyplot as plt
  plt.plot(np.arange(len(self.cost_his))*self.training_interval, self.cost_his)
  plt.ylabel('Training Loss')
  plot.xlabel('Time Frames')
  plt.show()

In [ ]:
def plot_rate(rate_his, rolling_intv=50):
  import matplotlib.pylot as plt
  import pandas as pd
  import matlplotlib as mpl
  rate_array = np.asarray(rate_his)

In [ ]:
df = pd.DataFrame(rae_his)